# Core Functionalities Checklist
- Includes data preprocessing (normalization, outlier removal, interpolation, redshift correction). DONE
- Facilitates metadata extraction (identifiers, coordinates, chemical abundances, redshifts, or other fields requested by end-user). DONE...?
- Alignment in wavelength for all the spectra across a predefined range, which might require interpolation.

This notebook is meant to be a way for us to get on the same page in terms of project goals and strategy

---

## Focus
The main goal of the software is to create a library that helps with astronomial research, SPECIFICALLY the classification between stars, galaxies and quasi-stellar objects (QSOs). To accomplish this, the library must interact with the Sloan Digital Sky Survery (SDSS) services, specifically their databases and APIs containing SPECTRAL data and related information.

## PP Master Strategy
1) SDSS services will be connected via astroquery.

Software must accept an ADQL query as a string format, which is a modified version of SQL. While there are some caveats about its additions, what we really care about is how we can implement it into our software. Luckily, astroquery is capable of running ADQL queries.

To my understanding, there seems to be two "major" types of searches done. 
One is your typical sql-like search:

<code>query = 
        SELECT TOP 10
            ra, dec, g, r
        FROM
            PhotoObj
        WHERE
            g > 18
result = sdss.launch_job(query=query)
result_table = result.get_results()</code>

The other does a search based of location:

<code>ra = 10.68458  # Example Right Ascension
dec = 41.26917  # Example Declination
search_radius = 0.1  # Example search radius in degrees
target_coords = coords.SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg))
result_table = ConeSearch.query_region(target_coords, radius=search_radius)</code>

2. In order to allow our searches to work, we need to understand the exact tables that are of interest to us in the SDSS (or at least the most relevant tables. Here's what I've dug up:

a) <code> SpecObj </code> : contains the spectral data and classificiaton for observed objects. It includes information about the spectroscopic observations such as spectral features and classifications (star, galaxy, and QSO) derived from the spectra. Sounds like it also contains redshifting data. This is likely the most important table we need to care about. However, there's a few more that we might want to think about as well.

b) <code> PhotoObj </code> : contains the photometric information for detected objects. It includes photometric data in multiple bands and may contain classification based on photometric data.

c) <code> Galaxy </code> : focuses mostly on galaxy data but contains columns to distinguish galaxies and other objects.

d) <code> Star </code> : focuses mostly on star data but contains columns that might be useful for classification.

e) <code> CrossID </code> : the glue that holds all of this stuff together. Contains information that can be used to relate objects between different catelogs.


3) At the bare minimum, I think we just need to bother with SpecObj.
Some important parameters:

<code>Redshift (z)</code>: Redshift is a fundamental parameter indicating the expansion of the universe or an object's relative velocity. It's crucial for estimating distances and studying cosmic evolution.

<code>Plate, FiberID,MJD</code>: These columns identify the specific plate, fiber ID, and modified Julian date associated with the spectroscopic observation. They are often used to trace back to the original observation data.

<code>Class</code>: The class column specifies the object's classification based on its spectrum. For quasars (QSOs), the class might be labeled as 'QSO'.

<code>Subclass</code>: Some datasets might include a subclassification of objects. For QSOs, this might further detail the type of quasar (e.g., 'BROADLINE', 'NARROWLINE', etc.). I don't thin we need to bother with this.

<code>Spectral Features</code>: Columns containing information about specific spectral features such as emission lines, absorption lines, or other characteristic spectral signatures


# tl;dr
1) Use astroquery to handle queries
2) Look at SpecObj for all relevant parameters.
3) Go crazy

---

# Data Acquisition

In [1]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords # query by coordinate
from astroquery.vo_conesearch import ConeSearch  # cone search within an object
from astropy import units as u

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

First, let's figure out what the heck we're looking at in SpecObj.

In [2]:
query = """SELECT TOP 10 *
FROM SpecObj"""

In [3]:
table = SDSS.query_sql(query)
spec_obj_cols = list(table.columns.keys())
print(spec_obj_cols)

['specObjID', 'bestObjID', 'fluxObjID', 'targetObjID', 'plateID', 'sciencePrimary', 'sdssPrimary', 'legacyPrimary', 'seguePrimary', 'segue1Primary', 'segue2Primary', 'bossPrimary', 'bossSpecObjID', 'firstRelease', 'survey', 'instrument', 'programname', 'chunk', 'platerun', 'mjd', 'plate', 'fiberID', 'run1d', 'run2d', 'tile', 'designID', 'legacy_target1', 'legacy_target2', 'special_target1', 'special_target2', 'segue1_target1', 'segue1_target2', 'segue2_target1', 'segue2_target2', 'boss_target1', 'eboss_target0', 'eboss_target1', 'eboss_target2', 'eboss_target_id', 'ancillary_target1', 'ancillary_target2', 'thing_id_targeting', 'thing_id', 'primTarget', 'secTarget', 'spectrographID', 'sourceType', 'targetType', 'ra', 'dec', 'cx', 'cy', 'cz', 'xFocal', 'yFocal', 'lambdaEff', 'blueFiber', 'zOffset', 'z', 'zErr', 'zWarning', 'class', 'subClass', 'rChi2', 'DOF', 'rChi2Diff', 'z_noqso', 'zErr_noqso', 'zWarning_noqso', 'class_noqso', 'subClass_noqso', 'rChi2Diff_noqso', 'z_person', 'class_per

/home/joon628/miniconda3/lib/python3.9/site-packages/numpy/lib/_iotools.py:680: RuntimeWarning: overflow encountered in conversion from string
  new_value = self.func(value)
/home/joon628/miniconda3/lib/python3.9/site-packages/numpy/lib/_iotools.py:672: RuntimeWarning: overflow encountered in conversion from string
  return self.func(value)
/home/joon628/miniconda3/lib/python3.9/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),


---


<code>SpecObjID</code>: Unique identifier for a spectrum observation.

<code>bestObjID, fluxObjID, targetObjID</code>: Different identifiers for objects used across different contexts in SDSS.

<code>plateID, run1d, run2d, mjd</code>: Information related to the spectroscopic plate, run, and modified Julian date associated with the observation.

<code>fiberID, spectrographID</code>: Identifiers associated with the fiber and spectrograph used for the observation.

<code>ra, dec, cx, cy, cz, xFocal, yFocal</code>: Coordinates and focal plane information related to the observed object.

<code>z, zErr, zWarning</code>: Redshift-related values, errors, and warnings associated with redshift estimation.

<code>class, subClass</code>: Classification and subclassification of objects based on their spectra.

<code>instrument, programname, survey</code>: Information about the survey, instrument used, and program name.

<code>special_target1, special_target2, legacy_target1, legacy_target2</code>: Target flags or identifiers used for specific selections or classifications.

<code>snMedian_u, snMedian_g, snMedian_r, snMedian_i, snMedian_z</code>: Median signal-to-noise ratio (SNR) in different photometric bands.

<code>spectroFlux_u, spectroFlux_g, spectroFlux_r, spectroFlux_i, spectroFlux_z</code>: Spectroscopic flux in different photometric bands.

<code>anyAndMask, anyOrMask</code>: Bitmasks indicating any and all flags for specific conditions.

---

Now, let's do a basic query to faciliate data preprocessing (normalization, outlier removal, interpolation, redshift correction) and some exploratory data analysis.

In [4]:
#tables from astroquery have units, which kind of mess with dataframes a lot, so let's just ditch em
def ditch_units(df):    
    for column in df.columns:
        if isinstance(df[column][0], u.quantity.Quantity):
            df[column] = df[column].apply(lambda x: x.value)
    return df

In [107]:
def query_to_df(query):
    table = SDSS.query_sql(query, data_release=17)
    df = table.to_pandas()
    df= ditch_units(df)
    return df

In [115]:
query = """
    SELECT TOP 1 *
 FROM SpecObj WHERE firstRelease = 'dr17' AND class = 'STAR'
"""

In [116]:
df = query_to_df(query)

/home/joon628/miniconda3/lib/python3.9/site-packages/numpy/lib/_iotools.py:680: RuntimeWarning: overflow encountered in conversion from string
  new_value = self.func(value)
/home/joon628/miniconda3/lib/python3.9/site-packages/numpy/lib/_iotools.py:672: RuntimeWarning: overflow encountered in conversion from string
  return self.func(value)
/home/joon628/miniconda3/lib/python3.9/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),


In [117]:
df['class'].unique() #we can see that there's only three classes

array([b'STAR'], dtype=object)

In [119]:
df[['specObjID', 'plate', 'mjd', 'fiberID', 'firstRelease']]

,specObjID,plate,mjd,fiberID,firstRelease
0,7220539695150946304,6413,56336,522,b'dr17'


Redshift correction seems to be really complicated. I think we're supposed to abuse packages. It seems like any type of cosmological calculation can be done using <code>astropy</code>. So let's use astropy!

In [10]:
from astropy.cosmology import FlatLambdaCDM # we are assuming the simplest cosmological mode

In [11]:
#galaxy brain redshift correction using the simplest cosmological model
def apply_redshift_correction(redshift_values, **kwargs):
    cosmo = FlatLambdaCDM(**kwargs)
    distances = cosmo.comoving_distance(redshift_values).to(u.Mpc)
    return distances

In [12]:
distances_corrected = apply_redshift_correction(df['z'], H0=72, Om0=0.28)
df['z_corrected'] = distances_corrected
df = ditch_units(df)

In [13]:
df.head()

,specObjID,ra,dec,z,class,plate,fiberID,mjd,z_corrected
0,7048316320836722688,15.988656,31.121303,0.475454,b'GALAXY',6260,665,56568,1769.312326
1,7423108218786652160,15.985467,31.107748,0.285623,b'GALAXY',6593,182,56270,1114.318873
2,7423103270984327168,16.161876,31.118433,-0.000122,b'STAR',6593,164,56270,-0.509461
3,7048316870592536576,16.040245,31.105689,0.348952,b'GALAXY',6260,667,56568,1340.429785
4,7423103545862234112,16.055123,31.114574,0.472915,b'GALAXY',6593,165,56270,1760.999035


---

In [14]:
#normalizes only the numerical value columns
def normalize(df, id_included=True, id_name = 'ObjID'):
    numerical_cols = df.select_dtypes(include=['number']).columns
    
    #we don't want to normalize the object IDs
    if id_included:
        col_norm = [col for col in numerical_cols if (id_name.casefold() not in col.casefold())]
    else:
        col_norm = numerical_cols
        
    scaler = StandardScaler()
    df[col_norm] = scaler.fit_transform(df[col_norm])
    return df

In [15]:
#removes outliers using Z-score stuff
def remove_outliers(df, threshold = 3, id_included=True, id_name = 'ObjID'):
    numerical_cols = df.select_dtypes(include=['number']).columns
    
    #we don't want to calculate Z-scores for the object IDs
    if id_included:
        col_rm = [col for col in numerical_cols if (id_name.casefold() not in col.casefold())]
    else:
        col_rm = numerical_cols
    
    z_scores = (df[col_rm] - df[col_rm].mean()) / df[col_rm].std()
    outliers = (z_scores > threshold).any(axis=1)
    df = df[~outliers]
    return df

In [16]:
#simple way to fill in missing numerical values
from sklearn.impute import SimpleImputer
def impute(df, id_included=True, id_name = 'ObjID'): 
    numerical_cols = df.select_dtypes(include=['number']).columns

    if id_included:
        numerical_cols = [col for col in numerical_cols if (id_name.casefold() not in col.casefold())]
    else:
        numerical_cols = numerical_cols
        
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df[numerical_cols] = imputer.fit_transform(df[numerical_cols])
    
    return df

In [17]:
#randomly remove values from a dataframe so we can test imputation
def remove_values(df, missing_ratio=0.3):
    random_cells = np.random.choice(df.index, size=int(len(df) * missing_ratio), replace=False)
    random_columns = np.random.choice(df.columns, size=int(len(df.columns) * missing_ratio), replace=False)
    for idx in random_cells:
        df.loc[idx, random_columns] = np.nan
    return df


In [18]:
df_norm = normalize(df)
print(df_norm.shape)
df_norm.head()


(100, 9)


,specObjID,ra,dec,z,class,plate,fiberID,mjd,z_corrected
0,7048316320836722688,-1.789812,1.726068,-0.106343,b'GALAXY',-0.288028,-0.118412,0.868454,0.048860
1,7423108218786652160,-1.794078,1.587154,-0.369858,b'GALAXY',3.644453,-2.260480,-1.203598,-0.380867
2,7423103270984327168,-1.558112,1.696655,-0.766519,b'STAR',3.644453,-2.340309,-1.203598,-1.112283
3,7048316870592536576,-1.720806,1.566053,-0.281947,b'GALAXY',-0.288028,-0.109543,0.868454,-0.232521
4,7423103545862234112,-1.700905,1.657108,-0.109867,b'GALAXY',3.644453,-2.335874,-1.203598,0.043406


In [19]:
df_norm_out = remove_outliers(df_norm)
print(df_norm_out.shape)
df_norm_out.head()

(91, 9)


,specObjID,ra,dec,z,class,plate,fiberID,mjd,z_corrected
0,7048316320836722688,-1.789812,1.726068,-0.106343,b'GALAXY',-0.288028,-0.118412,0.868454,0.048860
3,7048316870592536576,-1.720806,1.566053,-0.281947,b'GALAXY',-0.288028,-0.109543,0.868454,-0.232521
5,7048317970104164352,-1.786475,1.388709,-0.107868,b'GALAXY',-0.288028,-0.091803,0.868454,0.046501
9,7048327041075093504,-1.352405,1.270712,-0.516315,b'GALAXY',-0.288028,0.054550,0.868454,-0.639197
11,7048327315953000448,-1.545440,1.195972,1.752621,b'QSO',-0.288028,0.058984,0.868454,2.066903


In [20]:
df2 = df.copy()
df2 = remove_values(df2)
df2.columns[df2.isnull().any()].tolist()

['dec', 'z_corrected']

In [21]:
df2 = impute(df2)
print(df2.columns[df2.isnull().any()].tolist())
df2.head()

NameError: name 'KNNImputer' is not defined

---

## Spectral Stuff?
Alignment in wavelength for all the spectra across a predefined range, which might require interpolation.

In [120]:
spec = SDSS.get_spectra(plate=6413, fiberID=590, mjd=56336)


/home/joon628/miniconda3/lib/python3.9/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),


In [150]:
print(dir(spec[0][1]))
print(spec[0][3].columns)
print(spec[0][3].data['LINENAME'])

['_EXCLUDE', '_MASK', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_buffer', '_calculate_checksum', '_calculate_datasum', '_calculate_datasum_with_heap', '_char_encode', '_character_as_bytes', '_checksum', '_checksum_valid', '_clear_table_keywords', '_close', '_columns_type', '_compute_checksum', '_compute_hdu_checksum', '_data_loaded', '_data_needs_rescale', '_data_offset', '_data_replaced', '_data_size', '_data_type', '_datasum', '_datasum_valid', '_default_name', '_dump_coldefs', '_dump_data', '_encode_byte', '_ext_comment', '_extension', '_file', '_from_data', '_get_raw_data', '_get_tbdata', '_get_timestamp', '_has_data', '_hdu_registry', '_header', '_header_offset', '_header_str', '_ini

## Etc.

In [ ]:
# an example of using a not query to get some data.
pos = coords.SkyCoord('0h8m05.63s +14d50m23.3s', frame='icrs')
xid = SDSS.query_region(pos, radius='5 arcsec', spectro=True)
print(xid)   

       ra              dec        ...     specobjid      run2d
---------------- ---------------- ... ------------------ -----
2.02344596573482 14.8398237551311 ... 845594848269461504    26
